In [1]:
!python main.py --date 20210323 --flowmeter 2

   #           date_time  ...  Stopped(LGRS/N:20965846) EndOfFile(LGRS/N:20965846)
0  1 2021-03-23 09:54:01  ...                       NaN                        NaN
1  2 2021-03-23 09:54:03  ...                       NaN                        NaN
2  3 2021-03-23 09:54:05  ...                       NaN                        NaN
3  4 2021-03-23 09:54:07  ...                       NaN                        NaN
4  5 2021-03-23 09:54:09  ...                       NaN                        NaN

[5 rows x 6 columns]
                     flow
date_time                
2021-03-23 09:54:01   NaN
2021-03-23 09:54:03   NaN
2021-03-23 09:54:05   NaN
2021-03-23 09:54:07   NaN
2021-03-23 09:54:09   NaN
1
[12.120689655172422, 8.158974358974362, 7.480812641083514, 9.838509316770182, 5.157377049180296]
2
[7.400593471810104]
3
[8.524416135881047]
4
[7.073619631901818, 5.528767123287679, 8.566801619433171, 8.566801619433171]
5
[18.657657657656678]
6
[4.649456521739095]
7
[5.083333333333094, 17.987500